# Bike Redistribution (35 points)

The city of Los Angeles operates a bike-sharing program with **10 stations** spread across different neighborhoods. Throughout the day, commuters rent bikes from one station and return them to another, creating **imbalances in bike availability** across the network. Each station can hold at most **285 bikes** at any time.

Urban planners typically think about bike demand as occurring in **four periods** throughout the day, corresponding to **morning rush hour, afternoon, evening rush hour, and after hours**. The times for each period are:

- **8 AM - 12 PM** (Morning Rush Hour)
- **12 PM - 4 PM** (Afternoon)
- **4 PM - 8 PM** (Evening Rush Hour)
- **8 PM - 8 AM** (After Hours)

For simplicity, we will assume that **demand remains the same every day of the week**.

### Redistribution Process

To ensure that each station has enough bikes available at the **start of each period**, the city employs a **redistribution team** that uses trucks to pick up and move bikes between stations **just before the next period begins**. We assume that bikes are **moved instantaneously** before each shift.

### Redistribution Costs

The cost of redistribution depends on multiple factors—such as the number of trucks, travel distances, and truck capacity. To keep things simple, we assume that **redistribution costs are proportional to the total distance traveled** by the redistributed bikes throughout the day. 

> **Note:** The distance that bikes travel when rented by riders **does not contribute** to redistribution costs.

The goal of the redistribution team is to **minimize costs** while ensuring that each station has enough bikes at the **start of each period** to meet its demand.

### Data and the "Pre" Notebook

You are provided with a **"pre" notebook** that loads the following data:

- **The demand matrix**, specifying the number of bike trips between each pair of stations in each time period.
- **The distance matrix**, providing the distance (in miles) between each pair of stations.

## Formulation
Below, I give a formulation for hte problem, but I haven't given verbal descriptions of each of the constraints.  (You should be able to deduce these from the description above.)

### Sets and Indices:
- $S$ : Set of stations, indexed by $i, j$ (where $|S| = 10$).  This is called ``stations" in the code below.
- $T = \{1, 2, 3, 4\}$ : Set of time periods, indexed by $t$.  Here we assume that $t=1$ corresponds to period 8 am - 12 pm, and $t=2$ corresponds to 12 pm - 4pm etc.  

### Parameters:
- $d_{i,j,t}$ : Expected number of trips from station $i$ to station $j$ during period $t$  This data is contained in the dictionary ``demands" below.
- $c_{i,j}$ : Distance (in miles) between station $i$ and station $j$  This data is contained in the dictionary "distances" below.
- $C$ : Maximum capacity of each station (285 bikes)  

### Decision Variables:
- $I_{i,t}$ : Number of bikes at station $i$ at the **start** of period $t$ for every $t$ in $T$
- $x_{i,j,t}$ : Number of bikes moved **from station $i$ to station $j$** before period $t$  for every $i$ in $S$, for every $j$ in $S$, and every $t$ in $T$.

### Formulation
$$
\min \sum_{t \text{ in } T} \sum_{i \text{ in } S} \sum_{j \text{ in } S} c_{i,j} x_{i,j,t}
$$
s.t.
$$
I_{i,t +1} = I_{i,t} + \sum_{j \text{ in } S} x_{j,i,t+1} - \sum_{j \text{ in } S} x_{i,j,t+1} - \sum_{j \text{ in } S} d_{i,j,t} + \sum_{j \text{ in } S} d_{j, i, t}
, 
\quad \text{for all } i \text{ in } S, \text{ and } t = 1, 2, 3
$$
$$
I_{i,1} = I_{i,4} + \sum_{j \text{ in } S} x_{j,i,1} - \sum_{j \text{ in } S} x_{i,j,1} - \sum_{j \text{ in } S} d_{i,j,4}, 
+ \sum_{j \text{ in } S } d_{j,i, 4}
\quad \text{for all } \text{ in } S.
$$
$$
I_{i,t} \geq \sum_{j \text{ in } S} d_{i,j,t}, \quad \text{for all } i \text{ in } S, t \text{ in } T
$$
$$
I_{i,t} \leq C, \quad \text{for all } i \text{ in } S, t \text{ in } T
$$
$$
x_{i,j,t} \geq 0, \quad I_{i,t} \geq 0, \quad \text{for all } i\text{ in } S, j \text{ in } S, t \text{ in } T
$$


### Q1) (10 points) Interpreting the formulation 
For each of the families of constraints above, provide a short verbal description of what the constraint means.  For clarity, there are 5 families of constraints (one for each line after the "s.t.").  So you should provide 5 short interpretations.  

## Data Wrangling
I did the data wrangling for you.

In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum
import pandas as pd

#read in stations.csv keeping only the first column and convert to a list
stations = pd.read_csv('stations.csv')
stations = stations.iloc[:, 0].tolist()


#read in distance_matrix.csv dropping the first row and first column
distance_matrix = pd.read_csv('distance_matrix.csv')
distance_matrix = distance_matrix.drop(distance_matrix.columns[0], axis=1)
distance_matrix = distance_matrix.values.astype(float)

#create a dictionary where keys are pairs of stations and values are distances
distances = {}
for i in range(len(stations)):
    for j in range(len(stations)):
        if i != j:
            distances[(stations[i], stations[j])] = distance_matrix[i][j]
        elif i == j:
            distances[(stations[i], stations[j])] = 0

#read in demand_matrix.csv dopping the header row
demand_matrix = pd.read_csv('demand_matrix.csv')

demands= {}
for i in range(len(stations)):
    for j in range(len(stations)):
        for k in range(1, 5):
            if i == j:
                demands[(stations[i], stations[j], k)] = 0.
            else:
                #identify the row where Origin == stations[i], Destination == stations[j], and Time Period Index == k
                row = demand_matrix[(demand_matrix['Origin'] == stations[i]) & (demand_matrix['Destination'] == stations[j]) & (demand_matrix['Time Period Index'] == k)]

                #assert that the row is not empty
                assert not row.empty, f"No demand found for {stations[i]} to {stations[j]} in time period {k}"

                #get the value of the Demand column
                demand = row['Demand'].values[0]

                #add the demand to the dictionary
                demands[(stations[i], stations[j], k)] = demand

#### Q2) (20 points) Implement the Above formulation in Gurobi and solve it.
Your code must run.  It should also print out 
 - the optimal objective value.  
 - the number of bikes at the beginning of each period at each station in a nice table

#### Q3) (5 points) Be Creative.  
Think about the above model.  We made a lot of simplifications and are ignoring a lot of features that are important in real bike sharing networks.  Pick one feature of the model that you think is too simplistic.  Describe the feature, explain why you think it is important, and propose how you would modify the model to account for this feature.  Your explanation should include a mathematical descriptoin of any additional variables/constraints that you would need.  You do not need to implement your new model, but you should be very clear about the required changes.  